In [46]:
import pandas as pd
import os
import glob
from tqdm import tqdm
import numpy as np
import cv2 as cv
from get_image_size import get_image_size

In [4]:
DATA_PATH = '/home/paperspace/data/oic_2018'
TEST_IMAGES_PATH = DATA_PATH + '/test'
VALID_IMAGES_PATH = DATA_PATH + '/valid'
TEST_ANNO_PATH = DATA_PATH + '/test-annotations-bbox.csv'
VALID_ANNO_PATH = DATA_PATH + '/valid-annotations-bbox.csv'
CLASS_INFO_CSV_MAP = DATA_PATH + '/challenge-2018-class-descriptions-500.csv'

In [7]:
class_df = pd.read_csv(CLASS_INFO_CSV_MAP, header=None)
class_df.columns = ['name','desc']
class_df['idx'] = range(1, len(class_df) + 1)
class_df = class_df.set_index('name')
class_df.to_csv(DATA_PATH + '/oic-2018-classes-500.csv')
class_df.head()

,desc,idx
name,,
/m/061hd_,Infant bed,1
/m/06m11,Rose,2
/m/03120,Flag,3
/m/01kb5b,Flashlight,4
/m/0120dh,Sea turtle,5


In [8]:
valid_anno_df = pd.read_csv(VALID_ANNO_PATH)
valid_anno_df.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,0001eeaf4aed83f9,freeform,/m/0cmf2,1,0.022464,0.964178,0.070656,0.800164,0,0,0,0,0
1,000595fe6fee6369,freeform,/m/02wbm,1,0.000000,1.000000,0.000233,1.000000,0,0,1,0,0
2,000595fe6fee6369,freeform,/m/02xwb,1,0.141030,0.180277,0.676262,0.732455,0,0,0,0,0
3,000595fe6fee6369,freeform,/m/02xwb,1,0.213781,0.253028,0.298764,0.354956,1,0,0,0,0
4,000595fe6fee6369,freeform,/m/02xwb,1,0.232926,0.288447,0.488954,0.545146,1,0,0,0,0


In [72]:
def format_csv(input_df, class_df, images_dir):
    image_shape_cache = pd.DataFrame(columns=['w','h'])
    output_df = input_df[['ImageID','LabelName','XMin','XMax','YMin','YMax']]
    output_df.columns = ['ImageID','class_label','xmin_p','xmax_p','ymin_p','ymax_p']
    output_df['image_name'] = output_df['ImageID'] + '.jpg'
    output_df  = output_df.drop(['ImageID'], axis=1)
    
    for fn in tqdm(set(output_df['image_name'])):
        (w, h) = get_image_size(images_dir + '/' + fn)
        image_shape_cache = image_shape_cache.append({'image_name': fn,
                                                      'w': w, 
                                                      'h' : h}, ignore_index=True)
    image_shape_cache = image_shape_cache.set_index('image_name')
    output_df['w'] = output_df['image_name'].map(image_shape_cache['w'])
    output_df['h'] = output_df['image_name'].map(image_shape_cache['h'])
    output_df['xmin'] = (output_df['xmin_p'] * output_df['w']).astype('float').round().astype('int32')
    output_df['xmax'] = (output_df['xmax_p'] * output_df['w']).astype('float').round().astype('int32')
    output_df['ymin'] = (output_df['ymin_p'] * output_df['h']).astype('float').round().astype('int32')
    output_df['ymax'] = (output_df['ymax_p'] * output_df['h']).astype('float').round().astype('int32')
    
    output_df['class_id'] = output_df['class_label'].map(class_df['idx'])

    print('input {} rows'.format(len(input_df)))
    output_df = output_df.dropna()
    print('formatted {} rows'.format(len(output_df)))

    output_df['class_id'] = output_df['class_id'].astype('uint8')
    output_df = output_df[['image_name','class_id','xmin','xmax','ymin','ymax','class_label']]
    
    return output_df

In [73]:
slim_valid_anno_df = format_csv(valid_anno_df[:], class_df, images_dir=VALID_IMAGES_PATH)
slim_valid_anno_df.to_csv(DATA_PATH + '/valid_anno_ssd_labels.csv', index=False)
slim_valid_anno_df.head()


/Users/phu/anaconda/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
100%|██████████| 35925/35925 [03:49<00:00, 156.42it/s]


input 204621 rows
formatted 166905 rows


,image_name,class_id,xmin,xmax,ymin,ymax,class_label
0,0001eeaf4aed83f9.jpg,59,23,987,32,358,/m/0cmf2
2,000595fe6fee6369.jpg,48,144,185,461,499,/m/02xwb
3,000595fe6fee6369.jpg,48,219,259,203,242,/m/02xwb
4,000595fe6fee6369.jpg,48,239,295,333,371,/m/02xwb
5,000595fe6fee6369.jpg,48,251,297,451,488,/m/02xwb


In [77]:
test_anno_df = pd.read_csv(TEST_ANNO_PATH)
test_anno_df.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000026e7ee790996,freeform,/m/07j7r,1,0.071905,0.145346,0.206591,0.391306,0,1,1,0,0
1,000026e7ee790996,freeform,/m/07j7r,1,0.439756,0.572466,0.264153,0.435122,0,1,1,0,0
2,000026e7ee790996,freeform,/m/07j7r,1,0.668455,1.000000,0.000000,0.552825,0,1,1,0,0
3,000062a39995e348,freeform,/m/015p6,1,0.205719,0.849912,0.154144,1.000000,0,0,0,0,0
4,000062a39995e348,freeform,/m/05s2s,1,0.137133,0.377634,0.000000,0.884185,1,1,0,0,0


In [78]:
slim_test_anno_df = format_csv(test_anno_df, class_df, TEST_IMAGES_PATH)
slim_test_anno_df.to_csv(DATA_PATH + '/test_anno_ssd_labels.csv',index=False)
slim_test_anno_df.head()

/Users/phu/anaconda/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
100%|██████████| 108159/108159 [31:57<00:00, 56.41it/s]


input 625282 rows
formatted 509937 rows


,image_name,class_id,xmin,xmax,ymin,ymax,class_label
0,000026e7ee790996.jpg,30,74,149,159,301,/m/07j7r
1,000026e7ee790996.jpg,30,450,586,203,334,/m/07j7r
2,000026e7ee790996.jpg,30,684,1024,0,425,/m/07j7r
3,000062a39995e348.jpg,230,140,578,158,1024,/m/015p6
5,0000c64e1253d68f.jpg,149,0,997,0,30,/m/07yv9


In [79]:
test = pd.read_csv(DATA_PATH + '/test_anno_ssd_labels.csv')

test.head()

,image_name,class_id,xmin,xmax,ymin,ymax,class_label
0,000026e7ee790996.jpg,30,74,149,159,301,/m/07j7r
1,000026e7ee790996.jpg,30,450,586,203,334,/m/07j7r
2,000026e7ee790996.jpg,30,684,1024,0,425,/m/07j7r
3,000062a39995e348.jpg,230,140,578,158,1024,/m/015p6
4,0000c64e1253d68f.jpg,149,0,997,0,30,/m/07yv9
